In [4]:
%%bigquery users_devices
SELECT event_date, event_timestamp, user_pseudo_id, user_id, device.category as device_type,
    device.mobile_brand_name as device_brand,
    device.mobile_model_name as device_model,
    device.mobile_marketing_name as device_name,
    device.operating_system as device_os,
    device.operating_system_version as device_os_version,
    h.key as key,
    h.value.string_value as client_id,
    ev.key as event_key,
    ev.value.string_value as event_params
FROM `vertex-ai-sandbox-380023.analytics_250000498.events_20230327` 
left join unnest(user_properties) as h
left join unnest(event_params) as ev
where h.key = "client_id"
    and (ev.key = "an_la" or ev.key = "page_track")
    and event_date between format_date("%Y%m%d", date_sub(current_date(), interval 2 week))
    and format_date("%Y%m%d", date_sub(current_date(), interval 1 day))

Query is running:   0%|          |

Downloading:   0%|          |

In [5]:
import pandas as pd
import numpy as np
import prince
from lightfm import LightFM
from tqdm import tqdm 
import umap.umap_ as umap
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.cluster import KMeans
from kmodes.kprototypes import KPrototypes
import plotly.graph_objects as go
from sklearn.manifold import TSNE
from sklearn.preprocessing import PowerTransformer

In [3]:
users_clean = pd.read_parquet("users_clean.pq")

In [6]:
## cleaning data
print(f"the shape of the DF is {users_devices.shape}")
users = users_devices[~users_devices["client_id"].isna()].drop_duplicates()
print(f"the shape of the DF after non client_id removal is {users.shape}")

the shape of the DF is (2677822, 14)
the shape of the DF after non client_id removal is (2163851, 14)


In [7]:
non_use_mask = users["user_id"].isna()
non_users = users[non_use_mask].copy()
non_users["is_user"] = 0
users_1 = users[~non_use_mask].copy()
users_1["is_user"] = 1
size_df = users.shape[0]
print("prep_df was: ", size_df, "events")
del users
users = pd.concat([non_users, users_1])
if size_df == users.shape[0]:
    print("new prep_df size is correct and equals", size_df)
else:
    print("there is an error, new size is", size_df-prep_df.shape[0], "entries smaller")

prep_df was:  2163851 events
new prep_df size is correct and equals 2163851


In [11]:
users_s = users.drop(columns=["key", "user_id", "user_pseudo_id", "device_name"])

In [12]:
# making a key to connect needed columns later
users_s["key"] = users_s["event_timestamp"].astype("str") + users_s["client_id"]

In [13]:
# it is better to use PIVOT, however there is some issue with duplicating indecies: adding key, reseting index doesn't help
users_columns = users_s[users_s.drop(columns=["event_key", "event_params"]).columns]\
    .copy().drop_duplicates().reset_index(drop=True)
users_for_pivot = users_s[["key", "event_key", "event_params"]].copy().drop_duplicates().reset_index(drop=True)
users_title = users_for_pivot[users_for_pivot["event_key"] == "an_la"].drop_duplicates()
users_track = users_for_pivot[users_for_pivot["event_key"] == "page_track"].drop_duplicates()


In [16]:
users_title.event_params.unique()

array(['add-on:add item', 'get free galaxy buds2 worth gbp139:buy now',
       'suggested searches:Screen protector', ...,
       'home appliances:refrigerators:refrigeration buying guide',
       'cheer together:watch video', 'search history:Watch pro 5'],
      dtype=object)

In [20]:
tags = users_title.groupby(["event_params"]).count().reset_index().sort_values(by="key", ascending=False)

In [25]:
tags[tags["key"] > 20].shape[0]/tags.shape[0]

0.17040313662410778